In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter

# Load the Honda CSV file
honda_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/HONDA.csv')
print(f"Loaded {len(honda_data)} records from HONDA.csv")
honda_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'HONDA\s+CARS\s+INDIA\s+LTD,?\s*', '', text)
    text = re.sub(r'HONDA\s+MOTORCYCLE\s+AND\s+SCOOTER\s+INDIA\s+\(P\)\s+LTD,?\s*', '', text)
    text = re.sub(r'HONDA\s+MOTORCYCLE\s*&?\s*SCOOTER\s*\(?\s*I\s*\)?\s*P\s*L,?\s*', '', text)
    text = re.sub(r'HONDA\s+SIEL\s+CARS\s+INDIA\s+LTD,?\s*', '', text)
    text = re.sub(r'H\.M\.S\.I\.\s+PVT\.\s+LTD\.,?\s*', '', text)
    text = re.sub(r'HONDA\s*', '', text)
    text = re.sub(r'H\s+', '', text)  # H BRIO, H JAZZ, etc.
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
honda_data['clean_model'] = honda_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'STD', 'LIMITED', 'CARS']
filtered_models = honda_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary Honda model keywords
honda_model_keywords = [
    # Sedans
    "CITY", "CIVIC", "ACCORD", "AMAZE",
    
    # Hatchbacks
    "JAZZ", "BRIO",
    
    # SUVs/Crossovers
    "WR-V", "BR-V", "CR-V", "ELEVATE",
    
    # MPVs
    "MOBILIO",
    
    # Two-Wheelers (present in data)
    "ACTIVA", "DIO", "AVIATOR", "UNICORN", "SHINE", "DREAM", "CLIQ", "HORNET",
    
    # Trim levels
    "VX", "SX", "VMT", "SMT", "EMT", "CVT", "GXI", "EXI", "ZX",
    
    # Engine types
    "VTEC", "I-VTEC", "IVTEC", "I-DTEC", "IDTEC",
    
    # Common model combinations
    "CITY ZX", "CITY VX", "CITY SV", "CITY EXI", "CITY GXI",
    "AMAZE VX", "AMAZE SX", "AMAZE EMT", "AMAZE SMT",
    "JAZZ VX", "JAZZ SV", "JAZZ SMT",
    "CIVIC VX", "CIVIC EMT"
]

# Create alias map for common misspellings and variations
honda_alias_map = {
    # City variations
    "CITI": "CITY",
    "SITY": "CITY",
    "CTY": "CITY",
    "CITTY": "CITY",
    "H ONDACITY": "CITY",
    "HOND CITY": "CITY",
    "HONA CITY": "CITY",
    
    # Amaze variations
    "AMAZZE": "AMAZE",
    "AMAZ": "AMAZE",
    "AMZE": "AMAZE",
    "AMASE": "AMAZE",
    "AMAZEE": "AMAZE",
    "AMAAZ": "AMAZE",
    
    # Jazz variations
    "JASS": "JAZZ",
    "JASZ": "JAZZ",
    "JAZS": "JAZZ",
    "JAAZ": "JAZZ",
    "JASZ": "JAZZ",
    "JAZZZ": "JAZZ",
    
    # Civic variations
    "CIVIK": "CIVIC",
    "CIVEC": "CIVIC",
    "CIVICK": "CIVIC",
    "CIVIC18": "CIVIC",
    "HODNA CIVIC": "CIVIC",
    "HOND CIVIC": "CIVIC",
    "HONDACIVIC": "CIVIC",
    
    # Accord variations
    "ACORDD": "ACCORD",
    "ACORD": "ACCORD",
    "ACOORD": "ACCORD",
    "ACORAD": "ACCORD",
    "ACOORD": "ACCORD",
    "ACCRD": "ACCORD",
    "ACOORD": "ACCORD",
    "HONDA ACOORD": "ACCORD",
    
    # WRV variations
    "WR V": "WR-V",
    "WRV": "WR-V",
    "W R V": "WR-V",
    "WR-V-": "WR-V",
    "WREV": "WR-V",
    "W-R-V": "WR-V",
    
    # BRV variations
    "BR V": "BR-V",
    "BRV": "BR-V",
    "B R V": "BR-V",
    "BR-V-": "BR-V",
    "B-R-V": "BR-V",
    
    # CRV variations
    "CR V": "CR-V",
    "CRV": "CR-V",
    "C R V": "CR-V",
    "CR-V-": "CR-V",
    "C-R-V": "CR-V",
    
    # Engine/transmission variations
    "I VTEC": "I-VTEC",
    "IVTEC": "I-VTEC",
    "I DTEC": "I-DTEC",
    "IDTEC": "I-DTEC",
    "I-V TEC": "I-VTEC",
    "I-VETC": "I-VTEC",
    "I-V-TEC": "I-VTEC",
    "I-D TEC": "I-DTEC",
    "I-D-TEC": "I-DTEC",
    "VETC": "VTEC",
    "VTEC": "VTEC",
    "DTECH": "DTEC",
    
    # Common combinations mapped to base models
    "CITY ZX": "CITY",
    "CITY VX": "CITY",
    "CITY SV": "CITY",
    "CITY VMT": "CITY",
    "CITY SMT": "CITY",
    "CITY EMT": "CITY",
    "CITY CVT": "CITY",
    "CITY GXI": "CITY",
    "CITY EXI": "CITY",
    
    "AMAZE VX": "AMAZE",
    "AMAZE SX": "AMAZE",
    "AMAZE SMT": "AMAZE",
    "AMAZE EMT": "AMAZE",
    "AMAZE CVT": "AMAZE",
    
    "JAZZ VX": "JAZZ",
    "JAZZ SV": "JAZZ",
    "JAZZ SMT": "JAZZ",
    "JAZZ EMT": "JAZZ",
    "JAZZ CVT": "JAZZ",
    
    "CIVIC VX": "CIVIC",
    "CIVIC EMT": "CIVIC",
    "CIVIC VMT": "CIVIC"
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 ]', '', text.upper())

# Function to extract Honda model
def extract_honda_model(text):
    txt = normalize(text)
    
    # Skip entries that clearly aren't Honda vehicles
    non_honda = ["TATA", "RENAULT", "MARUTI", "MAHINDRA"]
    for brand in non_honda:
        if brand in txt:
            return "NOT FOUND"
    
    # First, correct aliases
    for bad, good in honda_alias_map.items():
        if bad in txt:
            txt = txt.replace(bad, good)
    
    # Then search for any keyword
    for kw in sorted(honda_model_keywords, key=len, reverse=True):
        if kw in txt:
            return kw
    return "NOT FOUND"

# Apply extraction to the entire dataset
honda_data['honda_model'] = honda_data['rc_maker_model'].apply(extract_honda_model)

# Display extraction results
model_freq = honda_data['honda_model'].value_counts()
print("\nModel extraction results:")
print(model_freq.head(20))
print(f"\nModels found: {(honda_data['honda_model'] != 'NOT FOUND').sum()} out of {len(honda_data)} ({(honda_data['honda_model'] != 'NOT FOUND').sum()/len(honda_data)*100:.1f}%)")

# Create a final_model column using fallback logic
honda_data['final_model'] = honda_data['honda_model'].apply(lambda x: x if x != "NOT FOUND" else "NOT FOUND")

# Save results to CSV for review
output_path = '/home/bipin/Documents/kotak/car_model/clean_model/honda_models_extracted.csv'
honda_data.to_csv(output_path, index=False)
print(f"\nSaved results to {output_path}")

# Create a frequency mapping of original values to extracted models
mapping_df = honda_data.groupby('rc_maker_model')['honda_model'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else "NOT FOUND"
).reset_index()
mapping_df.columns = ['original_value', 'mapped_model']
mapping_df['count'] = mapping_df['original_value'].map(honda_data['rc_maker_model'].value_counts())
mapping_df = mapping_df.sort_values('count', ascending=False)

# Save the mapping for review
mapping_path = '/home/bipin/Documents/kotak/car_model/clean_model/honda_model_mapping.csv'
mapping_df.to_csv(mapping_path, index=False)
print(f"Saved mapping to {mapping_path}")

# Save the keywords and alias map for reuse
with open('/home/bipin/Documents/kotak/car_model/clean_model/honda_keywords.py', 'w') as f:
    f.write("honda_model_keywords = [\n")
    for kw in sorted(honda_model_keywords):
        f.write(f"    \"{kw}\",\n")
    f.write("]\n\n")
    
    f.write("honda_alias_map = {\n")
    for bad, good in sorted(honda_alias_map.items()):
        f.write(f"    \"{bad}\": \"{good}\",\n")
    f.write("}\n")

print("Keywords and alias map saved to honda_keywords.py")

Loaded 9293 records from HONDA.csv
Top model keywords by frequency:
1: 4820
MT: 3444
5: 2988
CITY: 2281
2: 1434
AMAZE: 1395
I: 1379
V: 1142
VX: 1022
I-VTEC: 983
S: 914
JAZZ: 742
VTEC: 703
DTEC: 703
I-DTEC: 680
CVT: 600
MOBILIO: 429
ZX: 352
SMT: 341
O: 329
SV: 316
E: 314
BRIO: 303
BS4: 265
AT: 262
ACTIVA: 255
VMT: 245
IVTEC: 245
CIVIC: 243
CAR: 227

Model extraction results:
honda_model
I-VTEC       2195
I-DTEC       1831
CITY         1401
MOBILIO       506
AMAZE         501
NOT FOUND     477
BRIO          375
ACTIVA        338
JAZZ          283
CIVIC         274
SHINE         205
CR-V          190
ACCORD        149
BR-V          112
DREAM          89
UNICORN        70
DIO            69
WR-V           63
AVIATOR        43
ELEVATE        23
Name: count, dtype: int64

Models found: 8816 out of 9293 (94.9%)

Saved results to /home/bipin/Documents/kotak/car_model/clean_model/honda_models_extracted.csv
Saved mapping to /home/bipin/Documents/kotak/car_model/clean_model/honda_model_mapping.csv